# LULC Maps of Mumbai: Classification and Georeferencing

Georeferencing and classifying pixels of land use maps of Mumbai

By Harsh Vardhan Pachisia

Jan - Feb 2024

In [12]:
# Creat new environment. Run this on command line
# conda create -n lulc numpy rasterio matplotlib pandas opencv jupyter scipy
# conda activate lulc

In [13]:
# setting up paths
user = 'harsh'
system = 'cpu'
if user == 'tom':
    root = '/Users/tombearpark/Library/CloudStorage/Dropbox/india_mortality/data'
    # Need to add path to the input PDF, I didn't see it on dropbox
    pdf_path = root + 'lulc/lulc_maps.pdf'
    output_folder = root + 'lucl/lulc_maps_tifs'
elif user == 'harsh' and system == 'cpu':
    #convert to dropbox equivalent
    root = '/Users/Harsh/Desktop/01_Projects/Mumbai_Flooding/'
    pdf_path = root + '/lulc_maps.pdf'
    #convert to dropbox equivalent
    output_folder = root
elif user== 'harsh' and system == 'dropbox':
    #convert to dropbox equivalent
    root = '/Users/harshpachisia/Library/CloudStorage/Dropbox/india_mortality/data'
    pdf_path = root + '/lulc_maps.pdf'
    #convert to dropbox equivalent
    output_folder = root + 'lucl/lulc_maps_tifs'

In [14]:
# importing packages
import os
import cv2
import numpy as np
import rasterio
from   rasterio.transform import from_origin
import matplotlib.pyplot as plt
import pandas as pd
from scipy.ndimage import distance_transform_edt

## PDF to TIFF Images

In [15]:
convert_images = False
if convert_images:
    from pdf2image import convert_from_path

    # Convert PDF to a list of images
    images = convert_from_path(pdf_path, dpi=600)

    years = [1972, 1977, 1991, 1998, 1999, 2000, 2001, 2002, 2003, 2005, 2009, 2011,
             2013, '2014_1', '2014_2', 2015]

    # Save each image as a TIFF
    for i, image in enumerate(images):
        filename = f'lulc_map_{years[i]}.tiff'
        image.save(f'{output_folder}{filename}', 'TIFF')

## Image Georeference and Classification

### Big picture steps:
1. Georeference the original image in QGIS
2. Classify the image of a map of land use in Mumbai (different colors with different land use)
3. Store that classification as a tif, define the boundaries
4. Import the classification data (classified image which has the geographical boundaries of Mumbai) onto QGIS
5. Import other data layers like Mumbai pincodes in QGIS
6. Do analysis of differences in pincodes.

### Image Georeferencing
For georeferencing the image (which means assigning real-world coordinates to the image), this generally involves:

1. Having control points that tie image pixel coordinates to geographic coordinates (latitude and longitude, for example).
2. Using these points to define a spatial reference system and transformation parameters.
3. Applying these parameters to the image to create a georeferenced image, typically saved as a GeoTIFF.

This was done in QGIS using the georeferencer. 

In [16]:
# manually do this based on  https://www.rapidtables.com/web/color/RGB_Color.html#color-picker
legend_colors = {
    # adding a background for the legend- to be removed later
    (227, 0, 5): 'Built up land',
    (13, 114, 78): 'Forest plantation',
    (2, 255, 198): 'Mangrove',
    (86, 255, 7): 'Vegetation',
    (255, 255, 5): 'Crop land/Grass land',
    (255, 254, 190): 'Fallow land',
    (209, 253, 123): 'Aquatic vegetation',
    (132, 1, 170): 'Marshy/Swampy land',
    (191, 255, 231): 'Mudflats',
    (200, 200, 200): 'Sandy Area',
    (252, 116, 223): 'Barren Land',
    (0, 77, 170): 'Waterbodies',
    (119, 0, 77): 'Others (Salt pans)',
    (255, 255, 255): 'Background'
}

# creating a color map for mapping the categories and pixels
# This is there to map to easier, more recognizable colors as compared to what
# was present in the original legend (for future work done for these maps)
color_map ={
 'Built up land': (255, 0, 0),
 'Crop land/Grass land': (255,255,0),
 'Fallow land': (255, 254, 190),
 'Vegetation': (0,255,0),
 'Forest plantation': (0,100,0),
 'Mangrove': (0,255,255),
 'Aquatic vegetation': (50,205,50),
 'Marshy/Swampy land': (128,0,128),
 'Mudflats': (173,216,230),
 'Sandy Area': (255,250,240),
 'Barren Land': (255,192,203),
 'Waterbodies': (0,0,128),
 'Others (Salt pans)': (119, 0, 77),
 'Background': (250,250,250)
}

## Image Classification

#### Steps to Classify Colors Based on a Legend:
1. Load the Image: Read the TIFF image into a format that allows access individual pixel colors.

2. Define the Color Legend: Define a mapping from the colors in the legend to their respective classes. Possible dictionary where keys are color tuples (in RGB format) and values are the corresponding class names or identifiers.

3. Process Each Pixel: Iterate over each pixel in the image and replace its color with the corresponding class identifier based on legend mapping.

4. Handling Ambiguity: Exact color matches will be rare due to variations in shading, lighting, or image quality. Need a method to handle colors that don't exactly match your legend colors, like finding the nearest color in your legend.

In [17]:
def load_image(geo_image_path):
    '''
    Function that loads the georeferenced image and saves the image dimensions
    to be used in later functions
    
    '''
    # Load the georeferenced image

    with rasterio.open(geo_image_path) as src:
        image = src.read()
        # Pixel sizes in the x and y dimensions
        pixel_size_x, pixel_size_y = src.res
        # Upper left coordinates
        upper_left_x, upper_left_y = src.transform * (0, 0)
        crs = src.crs

    return image, (pixel_size_x, pixel_size_y), (upper_left_x, upper_left_y), crs

In [18]:
def classify_pixels(pixels, legend_colors, tolerance=0):
    '''
    function to classify pixels using opencv with a tolerance

    Returns: matrix of classified pixels and counts of matches
    '''
    classified_pixels = np.empty(pixels.shape[:2], dtype=object)
    match_counts = {}
    for color, classification in legend_colors.items():
        lower_bound = np.array(color) - tolerance
        upper_bound = np.array(color) + tolerance
        mask = np.all(np.logical_and(pixels >= lower_bound, 
                                     pixels <= upper_bound), axis=-1)
        match_counts[classification] = np.sum(mask)

        classified_pixels[mask] = classification

    return classified_pixels, match_counts

### Tackling the None (black pixels) issue
We have a bunch of None's (black pixels) where no category was chosen. Hence, to get around this issue we assume that since the color of the pixel (given image quality was quite poor), the pixel next to the unclassified pixel must have a high chance of being that category. The idea is to search for the nearest pixel with a classification and assign that classification to the unclassified pixel. This can be done using a spatial search algorithm.

For example, an unclassified pixel next to built up area will be classified as built up area- we use a technique like the k-nearest neighbors algorithm, but optimizing it for this specific task. The Scipy library provides functions that can help with this, which can be used to find the nearest classified pixel. Check all the closest pixels, and give it the value of a very close pixel which also has a similar color?

In [19]:
def color_distance_matrix(rgb_image, classified_pixels, indices):
    """
    Helper function to compute a matrix of color distances between unclassified pixels and 
    their nearest classified neighbors.
    """
    unclassified_pixels = np.argwhere(classified_pixels == None)
    nearest_indices = (indices[0][unclassified_pixels[:, 0], unclassified_pixels[:, 1]],
                       indices[1][unclassified_pixels[:, 0], unclassified_pixels[:, 1]])

    # Extract RGB values for nearest classified pixels
    nearest_colors = rgb_image[nearest_indices[0], nearest_indices[1]]

    # Extract RGB values for unclassified pixels
    unclassified_colors = rgb_image[unclassified_pixels[:, 0], unclassified_pixels[:, 1]]

    # Calculate color distances
    return np.sqrt(np.sum((nearest_colors - unclassified_colors) ** 2, axis=1))


def fill_unclassified_pixels(classified_pixels, rgb_image, tolerance=0):
    '''
    Function that fills in the unclassified pixels.

    Returns a matrix of filled in pixels
    '''
    # Create a mask for unclassified pixels
    unclassified_mask = classified_pixels == None
    
    # Compute nearest neighbor indices for unclassified pixels
    distances, indices = distance_transform_edt(unclassified_mask, return_indices=True)

    # Compute color distances
    color_distances = color_distance_matrix(rgb_image, classified_pixels, indices)
    
    # Fill unclassified pixels
    filled_pixels = classified_pixels.copy()
    unclassified_positions = np.argwhere(unclassified_mask)
    for i, pos in enumerate(unclassified_positions):
        if color_distances[i] <= tolerance:
            filled_pixels[tuple(pos)] = classified_pixels[tuple(indices[:, pos[0], pos[1]])]
        else:
            # Fallback to the nearest spatial classification
            filled_pixels[tuple(pos)] = classified_pixels[tuple(indices[:, pos[0], pos[1]])]

    return filled_pixels

In [20]:
def convert_to_numeric(filled_classified_pixels):
    '''
    Function that converts the classified pixels to numeric to get back
    into a georeferenced image format and maps it

    Returns: A matrix of numeric classified pixels
    '''
    unique_classes = np.unique(
        filled_classified_pixels[filled_classified_pixels != None])
    class_to_id_map = {cls: i for i, cls in enumerate(unique_classes, start=1)}
    class_to_id_map[None] = 0
    numeric_classified_pixels = np.vectorize(
        class_to_id_map.get)(filled_classified_pixels)
    
    # convert to int8 since QGIS wont take int64
    numeric_classified_pixels = numeric_classified_pixels.astype(np.uint8)
    
    return numeric_classified_pixels

In [21]:
def save_image(output_path, numeric_classified_pixels, shape, crs, transform):
    '''
    Function that saves the georeferenced, classified image
    '''
    with rasterio.open(
        output_path, 'w', driver='GTiff',
        height=shape[0], width=shape[1], count=1, dtype=numeric_classified_pixels.dtype,
        crs=crs, transform=transform
    ) as dst:
        dst.write(numeric_classified_pixels, 1)

In [22]:
# Loop Over years where maps are available
years = [2005, 2009, 2013, 2014, 2015]

for year in years:
    geo_image_path = f"{root}/lulc/georeferenced_maps/{year}_geocoded.tif" # georeferenced image
    output_path = f"{root}/lulc/final_maps/lulc_{year}.tif" # output path for classified, georeferenced map

    # Load image
    image, pixel_sizes, upper_left_coords, crs = load_image(geo_image_path)
    rgb_image = np.dstack((image[0], image[1], image[2]))
    
    # classify pixels
    classified_pixels, match_counts = classify_pixels(rgb_image,
                                                       legend_colors,
                                                         tolerance=20)
    #deal with unclassified pixels
    filled_classified_pixels = fill_unclassified_pixels(classified_pixels, 
                                                        rgb_image)
    #conver to numeric
    numeric_classified_pixels = convert_to_numeric(filled_classified_pixels)
    transform = from_origin(upper_left_coords[0], upper_left_coords[1], 
                            pixel_sizes[0], pixel_sizes[1])
    # save the image
    save_image(output_path, numeric_classified_pixels, 
               rgb_image.shape[:2], crs, transform)

### Import into QGIS
Steps to take:
1. Load OSM boundaries and zoom into Mumbai
2. After importing georeferenced and classified image, apply style in properties.
3. Blur out background. 